In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import keras
from keras.preprocessing.text import Tokenizer

In [2]:
df = pd.read_csv("../data/processed/dataset.csv")

In [3]:
df.head()

,Unnamed: 0,Artist,Lyric
0,0,aaliyah,Mmm yeh yeh \n A special smile \n A certain to...
1,1,steven-tyler,"Right now, nothing else matters \n You and me ..."
2,2,the-beatles,"Gonna tell Aunt Mary 'bout Uncle John, \n He s..."
3,3,norah-jones,I'm lonely \n 'Cause I'm looking at pictures o...
4,4,dexys-midnight-runners,"No, I don't want sympathy, \n I just want some..."


Tokenization.

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["Lyric"].astype(str).str.lower())
total_words = len(tokenizer.word_index) + 1
tokenized_sentences = tokenizer.texts_to_sequences(df["Lyric"].astype(str))

Slash sequences into n gram sequence.

In [ ]:
def generate_sequences(tokenized_sentences):
    for i in tqdm(tokenized_sentences, desc="Generating sequences"):
        for t in range(1, len(i)):
            n_gram_sequence = i[: t + 1]
            yield n_gram_sequence

Pre-padding.

In [ ]:
sequence_generator = generate_sequences(tokenized_sentences)

# Find the maximum sequence length.
max_sequence_len = max(len(seq) for seq in tqdm(sequence_generator, desc="Calculating max sequence length"))

# Create a new generator for sequences.
sequence_generator = generate_sequences(tokenized_sentences)

# Pad sequences in smaller batches to save memory.
batch_size = 1000
padded_sequences = []

for batch in tqdm(iter(lambda: list(sequence_generator)[:batch_size], []), desc="Padding sequences"):
    padded_sequences.extend(keras.preprocessing.sequence.pad_sequences(batch, maxlen=max_sequence_len, padding="pre"))

input_sequences = np.array(padded_sequences)

Create predictors and labels.

In [ ]:
X, labels = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(labels, num_classes=total_words)

dataset = dict()
dataset["features"] = X
dataset["labels"] = y